In [1]:
from webScraper import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mikey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<font size="5">
    Step 1: Scrape Latest Articles
</font>

In [ ]:
# Scrape latest articles from https://nos.nl
# Outputs latest articles in 'articles_scrapped.json'
webDataScrape()

<font size="5">
    Step 2: Add to Master Article List
</font>

In [ ]:
# Updates master_article_list.json, using articles_scrapped.json as input
updateMasterList()

In [ ]:
# Check length of master article list
master_article_list = []
with open('master_article_list.json') as json_file:
    master_article_data = json.load(json_file)
    for article in master_article_data:
        master_article_list.append(Article(article['ID'], article['url'], article['title'], article['date_time'], article['text']))
print(len(master_article_list))

<font size="5">
    Step 3: Convert text in Articles to list of words with frequency
</font>

In [2]:
# Retrieve all articles from master list
master_article_list = []
with open('master_article_list.json') as json_file:
    master_article_data = json.load(json_file)
    for article in master_article_data:
        master_article_list.append(Article(article['ID'], article['url'], article['title'], article['date_time'], article['text']))

In [5]:
# Retrieve list of words from each article
full_word_list = []
for article in master_article_list:
    full_word_list.extend(article.retrieveWordList())
print(full_word_list)

['IN', 'BRUSSEL', 'HEEFT', 'DE', 'POLITIE', 'INGEGREPEN', 'OP', 'DE', 'EERSTE', 'DAG', 'VAN', 'DE', 'NATIONAL', 'CONSERVATISM', 'CONFERENCE', 'NATCON', "ZO'N", 'VEERTIG', 'CONSERVATIEVE', 'EN', 'RECHTSE', 'POLITICI', 'ZOUDEN', 'SPREKEN', 'OP', 'DIE', 'CONFERENTIE', 'ONDER', 'WIE', 'DE', 'BRITSE', 'MINISTER', 'SUELLA', 'BRAVERMAN', 'EN', 'DE', 'FRANSE', 'POLITICUS', 'ÉRIC', 'ZEMMOUR', 'BURGEMEESTER', 'VAN', 'DE', 'BRUSSELSE', 'DEELGEMEENTE', 'SINT-JOOST-TEN-NODE', 'EMIR', 'KIR', 'VERBOOD', 'HET', 'EVENEMENT', 'KORT', 'NA', 'DE', 'START', 'UIT', 'VREES', 'VOOR', 'DE', 'VERSTORING', 'VAN', 'DE', 'OPENBARE', 'ORDE', 'DE', 'POLITIE', 'GREEP', 'IN', 'TOEN', 'DE', 'BRITSE', 'POLITICUS', 'NIGEL', 'FARAGE', 'OP', 'HET', 'PODIUM', 'STOND', 'DE', 'GEESTELIJK', 'VADER', 'VAN', 'DE', 'BREXIT', 'DE', 'ZAAL', 'WERD', 'NIET', 'ONTRUIMD', 'MAAR', 'DE', 'POLITIE', 'LIET', 'GEEN', 'NIEUWE', 'BEZOEKERS', 'MEER', 'TOE', 'BEZOEKERS', 'DIE', 'NAAR', 'BUITEN', 'GINGEN', 'KONDEN', 'DAARNA', 'DE', 'ZAAL', 'NIET

In [7]:
word_list_count_dict = {}
for word in full_word_list:
    if word in word_list_count_dict.keys():
        word_list_count_dict[word] = word_list_count_dict[word] + 1
    else:
        word_list_count_dict[word] = 1
print(word_list_count_dict)
        


{'IN': 65, 'BRUSSEL': 4, 'HEEFT': 13, 'DE': 168, 'POLITIE': 5, 'INGEGREPEN': 2, 'OP': 37, 'EERSTE': 3, 'DAG': 4, 'VAN': 58, 'NATIONAL': 1, 'CONSERVATISM': 1, 'CONFERENCE': 2, 'NATCON': 1, "ZO'N": 3, 'VEERTIG': 1, 'CONSERVATIEVE': 2, 'EN': 40, 'RECHTSE': 1, 'POLITICI': 1, 'ZOUDEN': 2, 'SPREKEN': 1, 'DIE': 23, 'CONFERENTIE': 3, 'ONDER': 3, 'WIE': 2, 'BRITSE': 3, 'MINISTER': 2, 'SUELLA': 1, 'BRAVERMAN': 1, 'FRANSE': 1, 'POLITICUS': 2, 'ÉRIC': 1, 'ZEMMOUR': 1, 'BURGEMEESTER': 2, 'BRUSSELSE': 4, 'DEELGEMEENTE': 2, 'SINT-JOOST-TEN-NODE': 2, 'EMIR': 1, 'KIR': 1, 'VERBOOD': 1, 'HET': 65, 'EVENEMENT': 2, 'KORT': 3, 'NA': 4, 'START': 1, 'UIT': 8, 'VREES': 1, 'VOOR': 23, 'VERSTORING': 1, 'OPENBARE': 1, 'ORDE': 1, 'GREEP': 1, 'TOEN': 2, 'NIGEL': 1, 'FARAGE': 1, 'PODIUM': 1, 'STOND': 1, 'GEESTELIJK': 1, 'VADER': 1, 'BREXIT': 1, 'ZAAL': 2, 'WERD': 8, 'NIET': 23, 'ONTRUIMD': 1, 'MAAR': 17, 'LIET': 1, 'GEEN': 13, 'NIEUWE': 8, 'BEZOEKERS': 2, 'MEER': 13, 'TOE': 2, 'NAAR': 9, 'BUITEN': 1, 'GINGEN': 1, '